<a href="https://colab.research.google.com/github/mkprasadp/NLP/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download("all");
from nltk.tag import HiddenMarkovModelTagger
from nltk.corpus import treebank
nltk.download('treebank')
corpus = treebank.tagged_sents()
train_data = corpus[:3000]
test_data = corpus[3000:]
hmm_tagger = HiddenMarkovModelTagger.train(train_data)
sentance = "I Love NLP"
tokens = nltk.word_tokenize(sentance)
tagged_sentance = hmm_tagger.tag(tokens)
print(tagged_sentance)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
